# Лабораторная работа № 1 (5)

In [1]:
import json
import numpy as np
from prettytable import PrettyTable
import warnings
warnings.filterwarnings("ignore")

## Парсинг

### Реализация

In [2]:
class Constraint:

    def __init__(self, data) -> None:
        super().__init__()
        self.coefs = data['coefs']
        self.b = data['b']
        self.type = data['type']
        if self.type == 'eq':
            self.function = lambda values: np.sum(np.multiply(self.coefs, values)) == self.b
        if self.type == 'leq':
            self.function = lambda values: np.sum(np.multiply(self.coefs, values)) <= self.b
        if self.type == 'geq':
            self.function = lambda values: np.sum(np.multiply(self.coefs, values)) >= self.b


    def check(self, values):
        return self.function(values)


class Task:

    def __init__(self, path) -> None:
        super().__init__()
        data = json.loads(open(path).read())
        self.f = data['f']
        self.goal = data['goal']
        self.constraints = []
        self.was_goal_changed = False
        self.first_step_f = np.zeros(len(self.f))
        self.artificial_indexes = []
        for constraint in data['constraints']:
            self.constraints.append(Constraint(constraint))


    def get_column(self, x):
        column = []
        for i in self.constraints:
            column.append(i.coefs[x])
        return column


    def check_constraints(self, values):
        flag = True
        for constraint in self.constraints:
            flag &= constraint.check(values)

        return flag


    def evaluate(self, values):
        return np.sum(np.multiply(self.f, values))
    

    def show_task(self):
        print("Task:")
        string = ''
        was_shown = False
        for i in range(len(self.f)):
            if self.f[i] > 0:
                if (i != 0) and was_shown:
                    string += ' + '
                string += str(self.f[i]) + ' * X_' + str(i + 1)
                was_shown = True
            elif self.f[i] < 0:
                if (i != 0) and was_shown:
                    string += ' '
                string += str(self.f[i]) + ' * X_' + str(i + 1)
                was_shown = True
        string += ' -> ' + self.goal
        print(string)


    def show_constraints(self):
        print("Constraints:")
        for i in self.constraints:
            string = ''
            was_shown = False
            for j in range(len(i.coefs)):
                if i.coefs[j] > 0:
                    if (j != 0) and was_shown:
                        string += ' + '
                    string += str(i.coefs[j]) + ' * X_' + str(j + 1)
                    was_shown = True
                elif i.coefs[j] < 0:
                    if (j != 0) and was_shown:
                        string += ' '
                    string += str(i.coefs[j]) + ' * X_' + str(j + 1)
                    was_shown = True
            if i.type == 'leq':
                string += ' <= ' + str(i.b)
            elif i.type == 'geq':
                string += ' >= ' + str(i.b)
            else:
                string += ' = ' + str(i.b)
            print(string)

### Визуализация

In [3]:
task = Task('example3.json')
task.show_task()
task.show_constraints()

Task:
5000 * X_1 + 2500 * X_2 -> max
Constraints:
4 * X_1 + 1.5 * X_2 <= 24
1200 * X_1 + 150 * X_2 <= 6000
20 * X_1 + 20 * X_2 <= 200
1 * X_1 >= 2


## Приведение к канонической форме

### Реализация

In [4]:
def make_canonical_form(task):
    if task.goal == "min":
        task.f = np.multiply(task.f, -1)
        task.goal = "max"
        task.was_goal_changed = True

    auxiliary_ind = 0
    m = len(task.constraints)
    n = len(task.f)

    for constraint in task.constraints:
        if constraint.type == "geq":
            constraint.coefs = np.concatenate([constraint.coefs, np.zeros(auxiliary_ind), [-1]])
            constraint.function = lambda values: np.sum(np.multiply(constraint.coefs, values)) == constraint.b
            constraint.type = "eq"
            auxiliary_ind += 1
        if constraint.type == "leq":
            constraint.coefs = np.concatenate([constraint.coefs, np.zeros(auxiliary_ind), [1]])
            constraint.function = lambda values: np.sum(np.multiply(constraint.coefs, values)) == constraint.b
            constraint.type = "eq"
            auxiliary_ind += 1

    for constraint in task.constraints:
        constraint.coefs = np.concatenate([constraint.coefs, np.zeros(auxiliary_ind+n-len(constraint.coefs))])
        if constraint.b < 0:
            constraint.coefs = np.multiply(constraint.coefs, -1)
            constraint.b = np.multiply(constraint.b, -1)
    
    n = len(task.f) + auxiliary_ind
    task.first_step_f = np.concatenate([task.first_step_f, np.zeros(auxiliary_ind)])
    
    for i in range(m):
        need_basis_column = True
        for j in range(n):
            tmp = task.get_column(j)
            if tmp[i] == 1 and tmp.count(0) == m - 1 and need_basis_column:
                need_basis_column = False
        if need_basis_column:
            for j in range(m):
                if j == i:
                    task.constraints[j].coefs = np.concatenate([task.constraints[j].coefs, [1]])
                else:
                    task.constraints[j].coefs = np.concatenate([task.constraints[j].coefs, [0]])
                constraint.function = lambda values: np.sum(np.multiply(constraint.coefs, values)) == constraint.b
            
            task.first_step_f = np.concatenate([task.first_step_f, [1]])
            task.artificial_indexes.append(len(task.f) + auxiliary_ind)
            auxiliary_ind += 1

    task.f = np.concatenate([task.f, np.zeros(auxiliary_ind)])

### Визуализация

In [5]:
make_canonical_form(task)
task.show_task()
task.show_constraints()

Task:
5000.0 * X_1 + 2500.0 * X_2 -> max
Constraints:
4.0 * X_1 + 1.5 * X_2 + 1.0 * X_3 = 24
1200.0 * X_1 + 150.0 * X_2 + 1.0 * X_4 = 6000
20.0 * X_1 + 20.0 * X_2 + 1.0 * X_5 = 200
1.0 * X_1 -1.0 * X_6 + 1.0 * X_7 = 2


## Двухфазный симплекс-метод

### Реализация

In [6]:
class SimplexTable():

    def __init__(self, task) -> None:
        self.m = len(task.constraints) + 1
        self.n = len(task.f) + 1
        self.table = np.zeros([self.m, self.n])
        self.blocked_columns = task.artificial_indexes
        self.basis = []
        self.second_phase_f = task.f
        self.was_reversed = task.was_goal_changed
        self.answer_exists = 1
        self.print_precision = 2
        for i in range(self.m):
            if i == self.m - 1:
                self.table[i, 0] = 0
                for j in range(1, self.n):
                    self.table[i, j] = task.first_step_f[j - 1]
            else:
                for j in range(self.n):
                    if j == 0:
                        self.table[i, j] = task.constraints[i].b
                    else:
                        self.table[i, j] = task.constraints[i].coefs[j - 1]
            
            if i < self.m - 1:
                need_basis_column = True
                for j in range(1, self.n):
                    tmp = task.get_column(j - 1)
                    if tmp[i] == 1 and tmp.count(0) == self.m - 2 and need_basis_column:
                        need_basis_column = False
                        self.basis.append(j - 1)
                  

    def first_phase(self):
        if len(self.blocked_columns) == 0:
            return self.table
        
        for k in self.blocked_columns:
            main_column = k + 1
            main_row = self.find_row(main_column)

            self.basis[main_row] = main_column - 1
            new_table = np.zeros([self.m, self.n])

            for i in range(self.n):
                new_table[main_row, i] = self.table[main_row, i] / self.table[main_row, main_column]

            for i in range(self.m):
                if i == main_row:
                    continue

                for j in range(self.n):
                    new_table[i, j] = self.table[i, j] - self.table[i, main_column] * new_table[main_row, j]

            self.table = new_table

        while self.end(False):
            main_column = self.find_column(True, False)
            main_row = self.find_row(main_column)

            self.basis[main_row] = main_column - 1
            new_table = np.zeros([self.m, self.n])

            for i in range(self.n):
                new_table[main_row, i] = self.table[main_row, i] / self.table[main_row, main_column]

            for i in range(self.m):
                if i == main_row:
                    continue

                for j in range(self.n):
                    new_table[i, j] = self.table[i, j] - self.table[i, main_column] * new_table[main_row, j]

            self.table = new_table

        return self.table


    def second_phase(self):
        self.next_phase()
        answer_w = 0
        for i in self.blocked_columns:
            answer_w += self.variable_result()[i]

        if answer_w != 0:
            self.answer_exists = 0
        else:
            while self.end(True):
                main_column = self.find_column(False, True)
                main_row = self.find_row(main_column)
                
                if main_column - 1 == self.basis[main_row] or self.answer_exists == 2:
                    break
                
                self.basis[main_row] = main_column - 1
                new_table = np.zeros([self.m, self.n])

                for i in range(self.n):
                    new_table[main_row, i] = self.table[main_row, i] / self.table[main_row, main_column]

                for i in range(self.m):
                    if i == main_row:
                        continue

                    for j in range(self.n):
                        new_table[i, j] = self.table[i, j] - self.table[i, main_column] * new_table[main_row, j]
                    
                new_table[self.m - 1, 0] = 0
                for i in range(self.m - 1):
                    new_table[self.m - 1, 0] += new_table[i, 0] * self.second_phase_f[self.basis[i]]

                self.table = new_table

        return self.table


    def next_phase(self):
        for i in range(1, self.n):
            self.table[self.m - 1, i] = 0

        for i in range(1, self.n):
            if self.second_phase_f[i - 1] != 0:
                if not i - 1 in self.basis:
                    self.table[self.m - 1, i] += self.second_phase_f[i - 1]
                else:
                    for j in range(1, self.n):
                        if j != i and self.table[self.basis.index(i - 1), j] != 0:
                            self.table[self.m - 1, j] -= self.second_phase_f[i - 1] * self.table[self.basis.index(i - 1), j]

        self.table[self.m - 1, 0] = 0
        for i in range(self.m - 1):
            self.table[self.m - 1, 0] += self.table[i, 0] * self.second_phase_f[self.basis[i]]


    def end(self, search_max):
        flag = False
        if search_max:
            for i in range(1, self.n):
                if self.table[self.m - 1, i] > 0:
                    flag = True
                    break
        else:
            for i in range(1, self.n):
                if self.table[self.m - 1, i] < 0:
                    flag = True
                    break
        
        return flag


    def find_column(self, first_phase, search_max):
        for i in range(1, self.n):
            if first_phase or i - 1 not in self.blocked_columns:
                main_column = i
                break

        if search_max:
            for i in range(main_column, self.n):
                if (first_phase or i - 1 not in self.blocked_columns) and self.table[self.m - 1, i] > self.table[self.m - 1, main_column]:
                    main_column = i
        else:
            for i in range(main_column, self.n):
                if (first_phase or i - 1 not in self.blocked_columns) and self.table[self.m - 1, i] < self.table[self.m - 1, main_column]:              
                    main_column = i

        return main_column


    def find_row(self, main_column):
        main_row = 0
        for i in range(self.m - 1):
            if self.table[i, main_column] > 0:
                main_row = i
                break

        for i in range(main_row + 1, self.m - 1):
            if self.table[i, main_column] > 0 and (self.table[i, 0] / self.table[i, main_column] < self.table[main_row, 0] / self.table[main_row, main_column]):
                main_row = i

        if main_row == 0 and self.table[main_row, main_column] <= 0:
            self.answer_exists = 2

        return main_row
    

    def check_inf_answers(self):
        coefs_f = self.second_phase_f
        coefs = np.zeros(len(coefs_f))
        for i in range(len(coefs_f)):
            coefs[i] = coefs_f[i]

        for i in range(len(self.basis)):
            base = self.basis[i]
            for j in range(1, self.n):
                if j - 1 != base:
                    coefs[j - 1] -= self.table[i, j] * coefs_f[base]
                    
        for i in range(len(coefs_f)):
            if i not in self.basis and coefs[i] == 0 and i not in self.blocked_columns:
                self.answer_exists = 3


    
    def variable_result(self):
        result = []
        for i in range(1, self.n):
            if i - 1 in self.basis:
                result.append(self.table[self.basis.index(i - 1), 0])
            else:
                result.append(0)
        return result


    def print_simplex_table(self):  
        x = PrettyTable()
        names = ['Basis', 'b']
        for i in range(1, self.n):
            names.append(str(i))

        x.field_names = names

        for i in range(self.m):
            if i == self.m - 1:
                row = ['C']
            else:
                row = [self.basis[i] + 1]
            
            for j in range(0, self.n):
                row.append(round(self.table[i, j], self.print_precision))
            x.add_row(row)

        answer = np.sum(self.second_phase_f * self.variable_result())
        row = [answer, 'f']
        for j in range(1, self.n):
            row.append(self.second_phase_f[j - 1])
        x.add_row(row)

        answer_w = 0
        for i in self.blocked_columns:
            answer_w += self.variable_result()[i]
        row = [answer_w, 'f\'']
        for j in range(1, self.n):
            if j - 1 in self.blocked_columns:
                row.append(1)
            else:
                row.append(0)

        x.add_row(row)

        print(x)


    def print_result(self):
        if self.answer_exists == 1:
            self.check_inf_answers()
            
        print('Result:')
        if self.answer_exists == 1:
            variables = self.variable_result()
            string = ''
            for i in range(1, self.n):
                if i != 1:
                    string += ', '
                string += 'X_' + str(i) + ' = ' + str(round(variables[i - 1], self.print_precision))

            print(string)
            if self.was_reversed:
                print('f = ' + str(round(-1 * self.table[self.m - 1, 0], self.print_precision)))
            else:
                print('f = ' + str(round(self.table[self.m - 1, 0], self.print_precision)))
            
        elif self.answer_exists == 0:
            print('Answer does not exist')

        elif self.answer_exists == 2:
            print('Function is unbordered')   
            print('f = inf')
                    
        else:
            print('Infinity amount of answers. One of them:')
            inf_flag = False
            variables = self.variable_result()
            string = ''
            for i in range(1, self.n):
                if i != 1:
                    string += ', '
                if variables[i - 1] == float("inf") and (not inf_flag) and self.second_phase_f[i - 1] != 0:
                    inf_flag = True
                    inf_ind = i - 1
                string += 'X_' + str(i) + ' = ' + str(round(variables[i - 1], self.print_precision))
            print(string)

            if inf_flag:
                coef = self.second_phase_f[inf_ind]
                if self.was_reversed == (coef > 0):
                    print('f = -inf')
                else:
                    print('f = inf')
            else:
                if self.was_reversed:
                    print('f = ' + str(round(-1 * self.table[self.m - 1, 0], self.print_precision)))
                else:
                    print('f = ' + str(round(self.table[self.m - 1, 0], self.print_precision)))

### Визуализация

In [7]:
simplex_table = SimplexTable(task)
print("Before 1 phase:")
simplex_table.print_simplex_table()
simplex_table.first_phase()
print("Between phases:")
simplex_table.print_simplex_table()
simplex_table.second_phase()
print("After 2 phase:")
simplex_table.print_simplex_table()

Before 1 phase:
+-------+--------+--------+--------+-----+-----+-----+------+-----+
| Basis |   b    |   1    |   2    |  3  |  4  |  5  |  6   |  7  |
+-------+--------+--------+--------+-----+-----+-----+------+-----+
|   3   |  24.0  |  4.0   |  1.5   | 1.0 | 0.0 | 0.0 | 0.0  | 0.0 |
|   4   | 6000.0 | 1200.0 | 150.0  | 0.0 | 1.0 | 0.0 | 0.0  | 0.0 |
|   5   | 200.0  |  20.0  |  20.0  | 0.0 | 0.0 | 1.0 | 0.0  | 0.0 |
|   7   |  2.0   |  1.0   |  0.0   | 0.0 | 0.0 | 0.0 | -1.0 | 1.0 |
|   C   |  0.0   |  0.0   |  0.0   | 0.0 | 0.0 | 0.0 | 0.0  | 1.0 |
|  0.0  |   f    | 5000.0 | 2500.0 | 0.0 | 0.0 | 0.0 | 0.0  | 0.0 |
|  2.0  |   f'   |   0    |   0    |  0  |  0  |  0  |  0   |  1  |
+-------+--------+--------+--------+-----+-----+-----+------+-----+
Between phases:
+---------+--------+--------+--------+-----+-----+-----+--------+---------+
|  Basis  |   b    |   1    |   2    |  3  |  4  |  5  |   6    |    7    |
+---------+--------+--------+--------+-----+-----+-----+--------+---

## Тесты

### Тест 1

In [8]:
task1 = Task('example.json')
task1.show_task()
task1.show_constraints()
make_canonical_form(task1)
simplex_table1 = SimplexTable(task1)
simplex_table1.first_phase()
simplex_table1.second_phase()
simplex_table1.print_result()

Task:
1 * X_1 + 2 * X_2 + 3 * X_3 -> max
Constraints:
1 * X_1 <= 1
1 * X_1 + 1 * X_2 >= -2
1 * X_1 + 1 * X_2 + 1 * X_3 = 3
Result:
X_1 = 0, X_2 = 0, X_3 = 3.0, X_4 = 1.0, X_5 = 2.0
f = 9.0


### Тест 2

In [9]:
task2 = Task('example2.json')
task2.show_task()
task2.show_constraints()
make_canonical_form(task2)
simplex_table2 = SimplexTable(task2)
simplex_table2.first_phase()
simplex_table2.second_phase()
simplex_table2.print_result()

Task:
6 * X_1 + 5 * X_2 -> max
Constraints:
-3 * X_1 + 5 * X_2 <= 25
-2 * X_1 + 5 * X_2 <= 30
1 * X_1 <= 10
3 * X_1 -8 * X_2 <= 6
Result:
X_1 = 10.0, X_2 = 10.0, X_3 = 5.0, X_4 = 0, X_5 = 0, X_6 = 56.0
f = 110.0


### Тест 3

In [10]:
task3 = Task('example3.json')
task3.show_task()
task3.show_constraints()
make_canonical_form(task3)
simplex_table3 = SimplexTable(task3)
simplex_table3.first_phase()
simplex_table3.second_phase()
simplex_table3.print_result()

Task:
5000 * X_1 + 2500 * X_2 -> max
Constraints:
4 * X_1 + 1.5 * X_2 <= 24
1200 * X_1 + 150 * X_2 <= 6000
20 * X_1 + 20 * X_2 <= 200
1 * X_1 >= 2
Result:
X_1 = 3.6, X_2 = 6.4, X_3 = 0, X_4 = 720.0, X_5 = 0, X_6 = 1.6, X_7 = 0
f = 34000.0


### Тест 4

In [11]:
task4 = Task('example4.json')
task4.show_task()
task4.show_constraints()
make_canonical_form(task4)
simplex_table4 = SimplexTable(task4)
simplex_table4.first_phase()
simplex_table4.second_phase()
simplex_table4.print_result()

Task:
1 * X_1 + 1 * X_2 -> max
Constraints:
2 * X_1 + 1 * X_2 <= 4
1 * X_1 + 2 * X_2 >= 3
Result:
X_1 = 0, X_2 = 4.0, X_3 = 0, X_4 = 5.0, X_5 = 0
f = 4.0


### Тест 5

In [12]:
task5 = Task('example5.json')
task5.show_task()
task5.show_constraints()
make_canonical_form(task5)
simplex_table5 = SimplexTable(task5)
simplex_table5.first_phase()
simplex_table5.second_phase()
simplex_table5.print_result()

#28/5, 0, 0, 1/5, 12/5

Task:
1 * X_4 -1 * X_5 -> min
Constraints:
1 * X_1 + 1 * X_4 -2 * X_5 = 1
-2 * X_4 + 1 * X_5 = 2
1 * X_3 + 3 * X_4 + 1 * X_5 = 3
Result:
Infinity amount of answers. One of them:
X_1 = 5.6, X_2 = 0, X_3 = 0, X_4 = 0.2, X_5 = 2.4, X_6 = 0
f = -2.2


### Тест 6

In [13]:
task6 = Task('example6.json')
task6.show_task()
task6.show_constraints()
make_canonical_form(task6)
simplex_table6 = SimplexTable(task6)
simplex_table6.first_phase()
simplex_table6.second_phase()
simplex_table6.print_result()

Task:
1 * X_1 + 2 * X_2 + 3 * X_3 -> min
Constraints:
2 * X_1 + 2 * X_2 + 2 * X_3 <= 5
1 * X_1 + 3 * X_3 >= 20
3 * X_1 + 2 * X_2 >= 100
Result:
Answer does not exist


### Тест 7

In [14]:
task7 = Task('example7.json')
task7.show_task()
task7.show_constraints()
make_canonical_form(task7)
simplex_table7 = SimplexTable(task7)
simplex_table7.first_phase()
simplex_table7.second_phase()
simplex_table7.print_result()

Task:
1 * X_1 + 2 * X_2 -> max
Constraints:
3 * X_1 = 3
1 * X_1 + 2 * X_2 >= 5
Result:
Function is unbordered
f = inf


### Тест 8

In [15]:
task8 = Task('example8.json')
task8.show_task()
task8.show_constraints()
make_canonical_form(task8)
simplex_table8 = SimplexTable(task8)
simplex_table8.first_phase()
simplex_table8.second_phase()
simplex_table8.print_result()

Task:
1 * X_1 + 1 * X_2 + 2 * X_3 + 2 * X_4 -> max
Constraints:
3 * X_1 + 3 * X_2 = 6
4 * X_3 + 4 * X_4 >= 8
2 * X_3 + 2 * X_4 <= 8
Result:
Infinity amount of answers. One of them:
X_1 = 2.0, X_2 = 0, X_3 = 4.0, X_4 = 0, X_5 = 8.0, X_6 = 0, X_7 = 0, X_8 = 0
f = 10.0


### Тест 9

In [16]:
task9 = Task('example9.json')
task9.show_task()
task9.show_constraints()
make_canonical_form(task9)
simplex_table9 = SimplexTable(task9)
simplex_table9.first_phase()
simplex_table9.second_phase()
simplex_table9.print_result()

Task:
1 * X_1 + 1 * X_2 + 2 * X_3 + 2 * X_4 -> max
Constraints:
3 * X_1 + 3 * X_2 >= 6
4 * X_3 + 4 * X_4 >= 8
2 * X_3 + 2 * X_4 <= 8
Result:
Function is unbordered
f = inf
